In [60]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

In [61]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

%matplotlib inline

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [62]:
def longHold(series):
    return series['Close'].iloc[-1] - series['Close'].iloc[0]

def shortHold(series):
    return -1 * longHold(series)

efficiency = 0.5

def longGreedy(series):
    maxI = series['High'].values.argmax()
    minI = series['Low'].values.argmin()
    
    buyPrice = (series['Low'].iloc[minI] - series['Close'].iloc[0]) * efficiency + series['Close'].iloc[0]
    
    if minI < maxI:
        sellPrice = (series['High'].iloc[maxI] - buyPrice) * efficiency + buyPrice
    else:
        # wait for low nonetheless, sell at end
        sellPrice = series['Close'].iloc[-1]
        
    return sellPrice - buyPrice
    
def shortGreedy(series):
    maxI = series['High'].values.argmax()
    minI = series['Low'].values.argmin()
    
    sellPrice = (series['High'].iloc[maxI] - series['Close'].iloc[0]) * efficiency + series['Close'].iloc[0]
    
    if minI > maxI:
        buyPrice = sellPrice - efficiency * (sellPrice - series['Low'].iloc[minI])
    else:
        # short at high nonetheless, buy back at end
        buyPrice = series['Close'].iloc[-1]
        
    return sellPrice - buyPrice
        

In [64]:
WINDOW_SIZE = 45

if not os.path.exists("data_with_labels"):
        os.mkdir("data_with_labels")

for ticker in tickers:
    data = pd.read_csv(f'./data/{ticker}.csv')
    optimalPolicies = []
    for i in range(data.shape[0] - WINDOW_SIZE):
        window = data[i:i+WINDOW_SIZE]
        candidates = np.array([longHold(window), shortHold(window), longGreedy(window), shortGreedy(window)])
        optimalPolicies.append(np.argmax(candidates))
    data['policy'] = pd.Series(optimalPolicies)
    for col in ['High', 'Open', "Low", "Close", "Volume"]:
        data.drop(col, axis=1, inplace=True)
    data.dropna(inplace=True)
    data.to_csv(f'./data_with_labels/{ticker}_with_labels.csv') #write to csv file
    print(f"{ticker} policies computed")
    
    plt.hist(data['policy'])
    plt.show()
    
    
    

SyntaxError: invalid syntax (<ipython-input-64-b39c28a435a3>, line 2)